# Operators

## Operator precedence

[[ref0](https://en.cppreference.com/w/cpp/language/operator_precedence), [ref1](https://docs.python.org/3.4/library/operator.html), [ref2](https://www.tutorialspoint.com/python/python_basic_operators.htm)]


Precedence | C/C++ Operator | Description | Associativity
-- | -- | -- | --
1 | :: | Scope resolution | Left-to-right
2 | a++ a-- | Suffix/postfix increment and decrement
  | type() type{} | Functional cast
  | a() | Function call
  | a[] | Subscript
  | . -> | Member access
3 | ++a --a | Prefix increment and decrement | Right-to-left
 | +a -a | Unary plus and minus
 | ! ~ | Logical NOT and bitwise NOT
 | (type) | C-style cast
 | \*a | Indirection (dereference)
 | &a | Address-of
 | sizeof | Size-of
 | new new[] | Dynamic memory allocation
 | delete delete[] | Dynamic memory deallocation
4 | .* ->* | Pointer-to-member | Left-to-right
5 | a\*b a/b a%b | Multiplication, division, and remainder
6 | a+b a-b | Addition and subtraction
7 | << >> | Bitwise left shift and right shift
8 | <=> | Three-way comparison operator (since C++20)
9 | < <= | For relational operators < and ≤ respectively
| \> >= | For relational operators > and ≥ respectively
10 | == != | For relational operators = and ≠ respectively
11 | & | Bitwise AND
12 | ^ | Bitwise XOR (exclusive or)
13 | &#124; | Bitwise OR (inclusive or)
14 | && | Logical AND
15 | &#124;&#124; | Logical OR
16 | a?b:c | Ternary conditional | Right-to-left
  | throw | throw operator
  | = | Direct assignment (provided by default for C++ classes)
  | += -= | Compound assignment by sum and difference
  | \*= /= %= | Compound assignment by product, quotient, and remainder
  | <<= >>= | Compound assignment by bitwise left shift and right shift
  | &= ^= &vert;= | Compound assignment by bitwise AND, XOR, and OR
17 | , | Comma | Left-to-right



* For example, if you want to evauate adding two integers, please try compiling and running following source code.



``` C++
// for cout
#include <iostream>

// declare intention to use std namespace
using namespace std;

// program starting point
int main(int argn, char * argv[]){

    // present operation result
    cout << "1 + 2 = " << 1 + 2 << '\n';

    // indicate that the program succeeded
    return 0;
}
```




Sr.No. | Python Operator | Description
-- | -- | --
1 | ** | Exponentiation<br>(raise to the power)
2 | ~ + - | Complement, unary plus and minus<br>(method names for the last two are +@ and -@)
3 | * / % // | Multiply, divide, modulo and floor division
4 | + - | Addition and subtraction
5 | >> << | Right and left bitwise shift
6 | & | Bitwise 'AND'
7 | ^ &vert; | Bitwise exclusive `OR' and regular `OR'
8 | <= < > >= | Comparison operators
9 | <> == != |  Equality operators
10 | %= /= //= -= += \*= \*\*= | Assignment operators
11 | is is not | Identity operators
12 | in not in | Membership operators
13 | not or and | Logical operators



* To evauate adding two integers or lists in python, please try selecting following cell and press <kbd>Shift</kbd>+<kbd>Enter</kbd>.



In [ ]:
# adding two numbers
print(f'1 + 2 = {1 + 2}')

# adding two lists
print(f'[1] + [2] = {[1] + [2]}')



## Some of the notable operators

* Following operators may need some additional attention.

| language | operator | description | example |
|:--------:|:--------:|:-----------:| ------- |
|  both   |   `+=`   | add in place | <pre><code class="language-c++">int a=0;<br>a+=2;</code></pre> |

[comment]:<> (https://www.quackit.com/html/tags/html_code_tag.cfm)
[comment]:<> (https://stackoverflow.com/questions/4823468/comments-in-markdown)

* Let's consider following C/C++ source code.



``` C++
#include <iostream>
#include <cstdio>

int main(int argn, char * argv[]){

	int a = int(argv[0][0]);
	int b = int(argv[0][1]);
	int c = 0;

	printf("argv[0] = %s\n", argv[0]);
	printf("argv[1] = %s\n", argv[1]);
	printf("argv[2] = %s\n\n", argv[2]);

	printf("a, b, c = %d, %d, %d\n", a, b, c);

	b++;
	puts("\nb++");
	printf("a, b, c = %d, %d, %d\n", a, b, c);

	b+=a;
	puts("\nb+=a");
	printf("a, b, c = %d, %d, %d\n", a, b, c);

	c = a + b;
	puts("\nc = a + b");
	printf("a, b, c = %d, %d, %d\n", a, b, c);

	return 0;

}
```



* Following is a part of assembly file from `-Wa,-adhln=$(ASM)` option.

``` assembly
   1:hello.cpp     **** #include <iostream>
   2:hello.cpp     **** #include <cstdio>
   3:hello.cpp     **** 
   4:hello.cpp     **** int main(int argn, char * argv[]){
  31              		.loc 1 4 0
  32              		.cfi_startproc
  33 0000 55       		pushq	%rbp
  34              		.cfi_def_cfa_offset 16
  35              		.cfi_offset 6, -16
  36 0001 4889E5   		movq	%rsp, %rbp
  37              		.cfi_def_cfa_register 6
  38 0004 4883EC20 		subq	$32, %rsp
  39 0008 897DEC   		movl	%edi, -20(%rbp)
  40 000b 488975E0 		movq	%rsi, -32(%rbp)
   5:hello.cpp     **** 
   6:hello.cpp     **** 	int a = int(argv[0][0]);
  41              		.loc 1 6 0
  42 000f 488B45E0 		movq	-32(%rbp), %rax
  43 0013 488B00   		movq	(%rax), %rax
  44 0016 0FB600   		movzbl	(%rax), %eax
  45 0019 0FBEC0   		movsbl	%al, %eax
  46 001c 8945F4   		movl	%eax, -12(%rbp)
   7:hello.cpp     **** 	int b = int(argv[0][1]);
  47              		.loc 1 7 0
  48 001f 488B45E0 		movq	-32(%rbp), %rax
  49 0023 488B00   		movq	(%rax), %rax
  50 0026 4883C001 		addq	$1, %rax
  51 002a 0FB600   		movzbl	(%rax), %eax
  52 002d 0FBEC0   		movsbl	%al, %eax
  53 0030 8945F8   		movl	%eax, -8(%rbp)
   8:hello.cpp     **** 	int c = 0;
  54              		.loc 1 8 0
  55 0033 C745FC00 		movl	$0, -4(%rbp)
  55      000000
   9:hello.cpp     **** 
  10:hello.cpp     **** 	printf("argv[0] = %s\n", argv[0]);
  56              		.loc 1 10 0
  57 003a 488B45E0 		movq	-32(%rbp), %rax
  58 003e 488B00   		movq	(%rax), %rax
  59 0041 4889C6   		movq	%rax, %rsi
  60 0044 488D3D00 		leaq	.LC0(%rip), %rdi
  60      000000
  61 004b B8000000 		movl	$0, %eax
  61      00
  62 0050 E8000000 		call	printf@PLT
  62      00
  11:hello.cpp     **** 	printf("argv[1] = %s\n", argv[1]);
  63              		.loc 1 11 0
  64 0055 488B45E0 		movq	-32(%rbp), %rax
  65 0059 4883C008 		addq	$8, %rax
  66 005d 488B00   		movq	(%rax), %rax
  67 0060 4889C6   		movq	%rax, %rsi
  68 0063 488D3D00 		leaq	.LC1(%rip), %rdi
  68      000000
  69 006a B8000000 		movl	$0, %eax
  69      00
  70 006f E8000000 		call	printf@PLT
  70      00
  12:hello.cpp     **** 	printf("argv[2] = %s\n\n", argv[2]);
  71              		.loc 1 12 0
  72 0074 488B45E0 		movq	-32(%rbp), %rax
  73 0078 4883C010 		addq	$16, %rax
  74 007c 488B00   		movq	(%rax), %rax
  75 007f 4889C6   		movq	%rax, %rsi
  76 0082 488D3D00 		leaq	.LC2(%rip), %rdi
  76      000000
  77 0089 B8000000 		movl	$0, %eax
  77      00
  78 008e E8000000 		call	printf@PLT
  78      00
  13:hello.cpp     **** 
  14:hello.cpp     **** 	printf("a, b, c = %d, %d, %d\n", a, b, c);
  79              		.loc 1 14 0
  80 0093 8B4DFC   		movl	-4(%rbp), %ecx
  81 0096 8B55F8   		movl	-8(%rbp), %edx
  82 0099 8B45F4   		movl	-12(%rbp), %eax
  83 009c 89C6     		movl	%eax, %esi
  84 009e 488D3D00 		leaq	.LC3(%rip), %rdi
  84      000000
  85 00a5 B8000000 		movl	$0, %eax
  85      00
  86 00aa E8000000 		call	printf@PLT
  86      00
  15:hello.cpp     **** 
  16:hello.cpp     **** 	b++;
  87              		.loc 1 16 0
  88 00af 8345F801 		addl	$1, -8(%rbp)
  17:hello.cpp     **** 	puts("\nb++");
  89              		.loc 1 17 0
  90 00b3 488D3D00 		leaq	.LC4(%rip), %rdi
  90      000000
  91 00ba E8000000 		call	puts@PLT
  91      00
  18:hello.cpp     **** 	printf("a, b, c = %d, %d, %d\n", a, b, c);
  92              		.loc 1 18 0
  93 00bf 8B4DFC   		movl	-4(%rbp), %ecx
  94 00c2 8B55F8   		movl	-8(%rbp), %edx
  95 00c5 8B45F4   		movl	-12(%rbp), %eax
  96 00c8 89C6     		movl	%eax, %esi
  97 00ca 488D3D00 		leaq	.LC3(%rip), %rdi
  97      000000
  98 00d1 B8000000 		movl	$0, %eax
  98      00
  99 00d6 E8000000 		call	printf@PLT
  99      00
  19:hello.cpp     **** 
  20:hello.cpp     **** 	b+=a;
 100              		.loc 1 20 0
 101 00db 8B45F4   		movl	-12(%rbp), %eax
 102 00de 0145F8   		addl	%eax, -8(%rbp)
  21:hello.cpp     **** 	puts("\nb+=a");
 103              		.loc 1 21 0
 104 00e1 488D3D00 		leaq	.LC5(%rip), %rdi
 104      000000
 105 00e8 E8000000 		call	puts@PLT
 105      00
  22:hello.cpp     **** 	printf("a, b, c = %d, %d, %d\n", a, b, c);
 106              		.loc 1 22 0
 107 00ed 8B4DFC   		movl	-4(%rbp), %ecx
 108 00f0 8B55F8   		movl	-8(%rbp), %edx
 109 00f3 8B45F4   		movl	-12(%rbp), %eax
 110 00f6 89C6     		movl	%eax, %esi
 111 00f8 488D3D00 		leaq	.LC3(%rip), %rdi
 111      000000
 112 00ff B8000000 		movl	$0, %eax
 112      00
 113 0104 E8000000 		call	printf@PLT
 113      00
  23:hello.cpp     **** 
  24:hello.cpp     **** 	c = a + b;
 114              		.loc 1 24 0
 115 0109 8B55F4   		movl	-12(%rbp), %edx
 116 010c 8B45F8   		movl	-8(%rbp), %eax
 117 010f 01D0     		addl	%edx, %eax
 118 0111 8945FC   		movl	%eax, -4(%rbp)
  25:hello.cpp     **** 	puts("\nc = a + b");
 119              		.loc 1 25 0
 120 0114 488D3D00 		leaq	.LC6(%rip), %rdi
 120      000000
 121 011b E8000000 		call	puts@PLT
 121      00
  26:hello.cpp     **** 	printf("a, b, c = %d, %d, %d\n", a, b, c);
 122              		.loc 1 26 0
 123 0120 8B4DFC   		movl	-4(%rbp), %ecx
 124 0123 8B55F8   		movl	-8(%rbp), %edx
 125 0126 8B45F4   		movl	-12(%rbp), %eax
 126 0129 89C6     		movl	%eax, %esi
 127 012b 488D3D00 		leaq	.LC3(%rip), %rdi
 127      000000
 128 0132 B8000000 		movl	$0, %eax
 128      00
 129 0137 E8000000 		call	printf@PLT
 129      00
  27:hello.cpp     **** 
  28:hello.cpp     **** 	return 0;
 130              		.loc 1 28 0
 131 013c B8000000 		movl	$0, %eax
 131      00
  29:hello.cpp     **** 
  30:hello.cpp     **** }
 132              		.loc 1 30 0
 133 0141 C9       		leave
 134              		.cfi_def_cfa 7, 8
 135 0142 C3       		ret
 136              		.cfi_endproc

```

* Here, we may want to take a look at instructions around "`24:hello.cpp`".

``` assembly
  24:hello.cpp     ****     c = a + b;
 114                      .loc 1 24 0
 115 0109 8B55F4           movl    -12(%rbp), %edx
 116 010c 8B45F8           movl    -8(%rbp), %eax
 117 010f 01D0             addl    %edx, %eax
 118 0111 8945FC           movl    %eax, -4(%rbp)
```



* This indicates that `b = a + b` operation happens as follows.


1. load `a` value to `%eax`
1. add value at `%eax` to `b`

* `b+=a` operation means the following instructions : "add a value to b"

``` assembly
  20:hello.cpp     **** 	b+=a;
 100              		.loc 1 20 0
 101 00db 8B45F4   		movl	-12(%rbp), %eax
 102 00de 0145F8   		addl	%eax, -8(%rbp)
```

* Other compound operators such as `*=` and `%=` also indicate similar operation.

## Exercises

### 00: Bitwise operators

* Please complete following Markdown table with bitwise operators in python and/or C/C++

| bitwise operator | expected behavior | available in python | available in C/C++ |
|----------|-------------------|---------------------|--------------------|
|    &vert;     |    Or     |          y          |         y          |

### 01 : Bitwise operators : python

* Please write a python cell presenting examples of the table above in binary

### 02 : Bitwise operators : C/C++

* Please write a C/C++ program presenting examples of the table above
* Please consider using `<bitset>`

### 03* : Bitwise operators : Assembly

* This is an optional assignment
* Generate an Assembly file of the above C++
* Duplicate the first table and add one more column indicating assembly *opcode*s